In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

<h1 style="font-size:10">1. Leer datos</h1>

Cargar datos: Glass

Atributos:
1. Número de identificación: 1 a 214
2. RI: índice de refracción
3. Na: Sodio (unidad de medida: porcentaje en peso en el óxido correspondiente, al igual que los atributos 4-10)
4. Mg: magnesio
5. Al: Aluminio
6. Si: silicio
7. K: Potasio
8. Ca: calcio
9. Ba: bario
10. Fe: Hierro
11. Tipo de vidrio: (atributo de clase)
     1. building_windows_float_processed
     2. building_windows_non_float_processed
     3. vehicle_windows_float_processed
     4. vehicle_windows_non_float_processed (ninguno en esta base de datos)
     5. contenedores
     6. vajilla
     7. faros
    
    
Distribución de clases: (de un total de 214 instancias)

1. 163 Vidrio de ventana (ventanas de edificios y ventanas de vehículos)
     1. 87 flotador procesado
         - 70 ventanas de construcción
         - 17 ventanas de vehículos
     2. 76 procesado sin flotación
         - 76 ventanas de construcción
         - 0 ventanas de vehículos
2. 51 Vidrio no de ventana
     - 13 contenedores
     - 9 vajillas
     - 29 faros

In [3]:
glass = pd.read_csv('glass.csv')
glass.columns = ['id','ri','na','mg','al','si','k','ca','ba','fe','class']
print("Glass shape: ", glass.shape)

Glass shape:  (213, 11)


Cargar datos: Titanic

In [4]:
train_titanic = pd.read_csv('titanic_train.csv', sep=',')
test_t = pd.read_csv('titanic_test.csv', sep=',')
gender_sub = pd.read_csv('gender_submission.csv', sep=',')
gender_sub.PassengerId = gender_sub.PassengerId.astype(float)

train_titanic = train_titanic.replace('Q', 0)
train_titanic = train_titanic.replace('S', 1)
train_titanic = train_titanic.replace('C', 2)
train_titanic = train_titanic.replace('male', 0)
train_titanic = train_titanic.replace('female', 1)
train_titanic = np.array(train_titanic.loc[:,['Sex','Age','Fare','Embarked', 'Survived']])
train_titanic = train_titanic[~np.isnan(train_titanic).any(axis=1)]

print("Titanic Train Shape : ", train_titanic.shape)

test1_t = pd.merge(test_t, gender_sub,  how='left', on='PassengerId')
test1_t = test1_t.replace('Q', 0)
test1_t = test1_t.replace('S', 1)
test1_t = test1_t.replace('C', 2)
test1_t = test1_t.replace('male', 0)
test1_t = test1_t.replace('female', 1)

print("Titanic Test Shape : ", train_titanic.shape)

test_titanic = np.array(test1_t.loc[:,['Sex','Age','Fare','Embarked', 'Survived']])
test_titanic = test_titanic[~np.isnan(test_titanic).any(axis=1)]

titanic = np.concatenate((train_titanic, test_titanic), axis=0)
titanic = pd.DataFrame(data=titanic, columns=['Sex','Age','Fare','Embarked', 'Survived'])

print("Titanic Shape: ", titanic.shape)


Titanic Train Shape :  (712, 5)
Titanic Test Shape :  (712, 5)
Titanic Shape:  (1043, 5)


<h1 style="font-size:10">2. Normalizar datos</h1>

In [5]:
def Normalizar_Datos(data):
    print(data)
    tam = len(data.columns)
    y = data.iloc[:,-1]
    data = data.iloc[:,:-1]
    data = (data - data.mean(axis=0))/data.std(axis=0)
    data.insert(tam-1, "y", y, True) 
    return data

In [6]:
titanic_data = Normalizar_Datos(titanic)

      Sex   Age      Fare  Embarked  Survived
0     0.0  22.0    7.2500       1.0       0.0
1     1.0  38.0   71.2833       2.0       1.0
2     1.0  26.0    7.9250       1.0       1.0
3     1.0  35.0   53.1000       1.0       1.0
4     0.0  35.0    8.0500       1.0       0.0
...   ...   ...       ...       ...       ...
1038  1.0   3.0   13.7750       1.0       1.0
1039  1.0  37.0   90.0000       0.0       1.0
1040  1.0  28.0    7.7750       1.0       1.0
1041  1.0  39.0  108.9000       2.0       1.0
1042  0.0  38.5    7.2500       1.0       0.0

[1043 rows x 5 columns]


In [7]:
glass_data = Normalizar_Datos(glass)

      id       ri     na    mg    al     si     k    ca    ba    fe  class
0      2  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.00      1
1      3  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.00      1
2      4  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.00      1
3      5  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.00      1
4      6  1.51596  12.79  3.61  1.62  72.97  0.64  8.07  0.00  0.26      1
..   ...      ...    ...   ...   ...    ...   ...   ...   ...   ...    ...
208  210  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.00      7
209  211  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.00      7
210  212  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.00      7
211  213  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.00      7
212  214  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.00      7

[213 rows x 11 columns]


<h1 style="font-size:10">3. Exactitud - Accuracy</h1>

In [174]:
def Calcular_Accuracy(X, y, W_dict, b_dict):
    _, A_dict = Forward(X, W_dict, b_dict)
    A_L = A_dict["A%d"%(len(A_dict)-1)]
    result = A_L > 0.5
    result = np.logical_xor(np.logical_not(result), y)
    print("Y SHAPE")
    print(y.shape)
    #return np.sum(result) / y.shape[1]
    return np.sum(result)

<h1 style="font-size:10">4. KFolds - Multiclase</h1> 

In [10]:
def kFolds(data, k=3, clases=[0, 1]):
    list_clases = []
    list_indices_break = []
    list_folds_clases = []
    
    #ordenar segun ultima columna
    data = data.sort_values(by=["y"]).values

    for i in range(1, len(clases)):
        #obtener el indice de la primera ocucrrencia de unos
        indice_break = np.where(data[:,-1] == clases[i])[0][0]
        list_indices_break.append(indice_break)
        
    #dividar los datos en dos clases
    list_clases = np.split(data, list_indices_break)

    
    #dividir en k folds
    for clase in list_clases:
        clase = np.array_split(clase, k)
        list_folds_clases.append(clase)

    datos_split = list()

    #distribuir una porcion equitativa de cada clase a cada fold
    for i in range(k):
        fold = list_folds_clases[0][i]
        
        #combinar cada fold con una proporcion de cada clase
        for j in range(len(list_folds_clases)):
            fold = np.concatenate((fold, list_folds_clases[j][i]))

        #mezclar los dataFrames
        np.random.shuffle(fold)

        #separar x e y
        x_data = fold[:, :-1]
        y_data = fold[:, -1]

        datos_split.append([x_data, y_data])

    return datos_split

In [61]:
titanic_folds = kFolds(titanic_data, k=3, clases=[0,1])

(96, 10)
(94, 10)
(92, 10)


In [50]:
glass_folds = kFolds(glass_data, k=3, clases=[1,2,3,5,6,7])

<h1 style="font-size:10">5. Sigmoidal</h1> 

In [8]:
def Sigmoidal(x):
    return 1.0 / (1.0+np.exp(-x))

In [189]:
def Activation(X, W, b):
    print(type(X))
    print(type(W))
    print(X.shape)
    print(W.shape)
    
    Z = np.dot(X,W)  + b
    #Z = np.mat(X) * np.mat(W)+ b
    A = Sigmoidal(Z)
    return A, Z

<h1 style="font-size:10">6. Función de Costo</h1> 

In [144]:
def Calcular_Funcion_Costo(X, y):
    c = np.sum( (y * np.log(X)) + ((1-y) * np.log(1-X)) )
    costo = (-c)/y.shape[1]
    return costo

<h1 style="font-size:10">7. Derivada Sigmoidal</h1> 

In [17]:
def dSigmoidal(D):
    return D * (1.0 - D)

<h1 style="font-size:10">8. Forward</h1> 

In [157]:
def Forward(X, W, b):
    A = X
    Z_dict = {}
    A_dict = {"A0":A}
    for l in range(len(W)):
        A, Z = Activation( A, W[ "W%d"%(l+1) ], b[ "b%d"%(l+1) ] )
        Z_dict["Z%d"%(l+1)] = Z
        A_dict["A%d"%(l+1)] = A
    return Z_dict, A_dict

<h1 style="font-size:10">9. Backward</h1> 

In [158]:
def Backward(W, b, z, A, y, learn_rate):
    m = A["A0"].shape[1]
    dA = get_dC_dA_L(A["A%d"%(len(A)-1)], y)
    for l in reversed(range(len(W))):
        dZ = np.multiply(dA, dS(z["Z%d"%(l+1)]))
        dW = np.divide(np.dot( dZ, A["A%d"%(l)].T ), m)
        db = np.divide(np.sum( dZ, axis=1, keepdims=True ), m)
        dA = np.dot( W["W%d"%(l+1)].T, dZ )
        W["W%d"%(l+1)] = W["W%d"%(l+1)] - np.multiply(learn_rate, dW)
        b["b%d"%(l+1)] = b["b%d"%(l+1)] - np.multiply(learn_rate, db)
    return W, b

<h1 style="font-size:10">10. Gradiente descendiente</h1> 

In [159]:
def Gradiente_Descendiente(X, y, W, b, iterations, learn_rate):
    cost_history = np.zeros(iterations)
    for i in range(iterations):
        Z_dict, A_dict = Forward(X, W, b)
        W, b = Backward(W, b, Z_dict, A_dict, y, learn_rate)
        cost_history[i] = Calcular_Funcion_Costo(A_dict["A%d"%(len(A_dict)-1)], y)
    return W, b, cost_history


<h1 style="font-size:10">11. Funciones auxiliares</h1> 

In [160]:
def generate_W(cur, prev):
    return np.random.rand(cur, prev)
def generate_b(b):
    return np.random.rand(b, 1)


def generator_W_b(layers):
    W = {}
    b = {}
    for l in range(len(layers)-1):
        W["W%d"%(l+1)] = generate_W(layers[l], layers[l+1])
        b["b%d"%(l+1)] = generate_b(layers[l+1])
    return W, b

def getNeurons(num_layers=1, neurons=[1]):
    lays = []
    for i in range(num_layers):
        lays.append(random.choice(neurons))
    return lays

<h1 style="font-size:10">12. Experimentos</h1> 

In [190]:
#tasas = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
#iteraciones = [500, 1000, 1500, 2000, 2500, 3000, 3500]
tasas = [0.01]
iteraciones = [500]
num_hidden_layers = [1,2,3,4]
num_neurons_per_l = [3,4,5]

numFolds = len(titanic_folds)
results_titanic = {}
weights_titanic = {}
matrix_ac = np.random.rand(len(tasas),len(iteraciones),numFolds)

for l in num_hidden_layers:
    lay = getNeurons(l,num_neurons_per_l)
    for i in range(numFolds):
        for t in range(len(tasas)):
            acc_test_total = 0.0
            tasa = tasas[t]
            print(len(iteraciones))
            for j,it in enumerate(iteraciones):
                print("Entra")
               
            
                #actual fold (usado para test)
                fold = titanic_folds[i]
                test_fold_X = fold[0]
                test_fold_y = fold[1]

                #los otros dos folds usados para train
                fold_2 = titanic_folds[(i+1)%numFolds]
                fold_3 = titanic_folds[(i+2)%numFolds]
                train_fold_X = np.concatenate((fold_2[0], fold_3[0]), axis=0)
                train_fold_y = np.concatenate((fold_2[1], fold_3[1]), axis=0)

                act_layers = []

                act_layers.append(train_fold_X.shape[1]) 
                act_layers.append(len(lay))
                act_layers.append(1)

                weigths, bias = generator_W_b(act_layers)
                weigths_r, bias_r, cost_history_r = Gradiente_Descendiente(train_fold_X, train_fold_y, weigths, bias, j, tasa)
                acc_test = Calcular_Accuracy(test_fold_X,test_fold_y, weigths_r, bias_r)
                acc_test_total += acc_test
                
                
                
                '''
                weigths = weights_generator(act_layers)
                activations = Generar_Activaciones(act_layers)
                print(train_fold_y)
                weights_r, cost_history_r = Gradiente_Descendiente(train_fold_X, train_fold_y, weigths, it, t, activations)
               
                matrix_ac[t,j,i] = Calcular_Accuracy(test_fold_X,test_fold_y,new_Weigths,activations)
                
                
                print(len(cost_history_r))
                costo_gradiente = cost_history_r[-1]
                
                results_titanic["tasa_"+str(tasa)+"_iter_"+str(iteracion)+"_fold_"+str(i)] = costo_gradiente
                weights_titanic["tasa_"+str(tasa)+"_iter_"+str(iteracion)+"_fold_"+str(i)] = weights_r
                '''

1
Entra
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(559, 4)
(4, 1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(559, 1)
(1, 1)
Y SHAPE
(559,)
1
Entra
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(556, 4)
(4, 1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(556, 1)
(1, 1)
Y SHAPE
(556,)
1
Entra
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(556, 4)
(4, 1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(556, 1)
(1, 1)
Y SHAPE
(556,)
1
Entra
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(559, 4)
(4, 2)


ValueError: operands could not be broadcast together with shapes (559,2) (2,1) 

In [124]:
print(weights_titanic)

{}


# SVM

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm

# The Dataset comes from:
# https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

def load(path_test, path_train):
  # Load up the data.
  with open(path_test, 'r')  as f: testing  = pd.read_csv(f)
  with open(path_train, 'r') as f: training = pd.read_csv(f)

  # The number of samples between training and testing can vary
  # But the number of features better remain the same!
  n_features = testing.shape[1]

  X_test  = testing.iloc[:, :n_features-1]
  X_train = training.iloc[:, :n_features-1]
  y_test  = testing.iloc[:,n_features-1].values.ravel()
  y_train = training.iloc[:,n_features-1].values.ravel()

  return X_train, X_test, y_train, y_test


def linear_SVM(X_train, y_train, X_test, y_test):

    svc = svm.SVC(kernel='linear', C=1, gamma=0.001)
    svc.fit(X_train, y_train) 
    
    # Calcular el score del SVM
    score = svc.score(X_test, y_test)
    return score

def poly_SVM(X_train, y_train, X_test, y_test):

    svc = svm.SVC(kernel='poly', C=1, degree=3, gamma=0.001)
    svc.fit(X_train, y_train) 
    
    # Calcular el score del SVM
    score = svc.score(X_test, y_test)
    return score

def sigmoid_SVM(X_train, y_train, X_test, y_test):

    svc = svm.SVC(kernel='sigmoid', C=1, gamma=0.001)
    svc.fit(X_train, y_train) 
    
    # Calcular el score del SVM
    score = svc.score(X_test, y_test)
    return score

def kernel_gausiano_SVM(X_train, y_train, X_test, y_test):

    svc = svm.SVC(kernel='rbf', C=1, gamma=0.001)
    svc.fit(X_train, y_train) 
    
    # Calcular el score del SVM
    score = svc.score(X_test, y_test)
    return score




#cargando datos
X_train, X_test, y_train, y_test = load('Datasets/optdigits.tes', 'Datasets/optdigits.tra')

score = linear_SVM(X_train, y_train, X_test, y_test)
print ("Score de linear SVM:\n", score)

score = poly_SVM(X_train, y_train, X_test, y_test)
print ("Score de polynomial SVM:\n", score)

score = sigmoid_SVM(X_train, y_train, X_test, y_test)
print ("Score de sigmoid SVM:\n", score)

score = kernel_gausiano_SVM(X_train, y_train, X_test, y_test)
print ("Score de kernel gausiano SVM:\n", score)

Score de linear SVM:
 0.9610244988864143
Score de polynomial SVM:
 0.9749443207126949
Score de sigmoid SVM:
 0.717706013363029
Score de kernel gausiano SVM:
 0.982739420935412
